In [156]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.optimize import minimize

In [157]:
#加载数据
df = pd.read_excel('各参数SEA.xlsx')
# print(df.sample(10))

In [158]:
#描述性统计
print(df.describe())
print(sea_range := df['SEA'].max() - df['SEA'].min())

             α(°)       β(°)       l1/l2        t_mm          SEA
count  560.000000  560.00000  560.000000  560.000000   560.000000
mean    45.000000   55.00000    2.250000    1.150000   669.679696
std     10.008941    7.07739    0.559517    0.111903   151.469724
min     30.000000   45.00000    1.500000    1.000000   368.340000
25%     35.000000   50.00000    1.875000    1.075000   544.565000
50%     45.000000   55.00000    2.250000    1.150000   684.160000
75%     55.000000   60.00000    2.625000    1.225000   785.387500
max     60.000000   65.00000    3.000000    1.300000  1023.070000
654.73


In [159]:
#数据拆分
X_train, X_test, y_train, y_test = train_test_split(
    np.array(df[['α(°)',  'β(°)',  'l1/l2',  't_mm',]]),
    np.array(df[['SEA']]),
    test_size=0.2,
    shuffle=True,
)

In [160]:
#训练多项式回归模型
model = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=4)),
    ('linear', RidgeCV(alphas=np.logspace(-3, 3, 6, base=10), cv=10),),
])
model.fit(X_train, y_train)

,steps,"[('scaler', ...), ('poly', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,degree,4
,interaction_only,False
,include_bias,True
,order,'C'


In [161]:
#模型评估
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2_test = r2_score(y_test, y_pred)
r2_train = r2_score(y_train, model.predict(X_train))
print(rmse)
print(r2_test)
print(r2_train)

22.348125574579615
0.9750979541104652
0.9877916451784351


In [162]:
#优化
##定义目标函数
def objective(x):
    x = np.array(x).reshape(1, -1)
    y = model.predict(x)
    return -y

In [163]:
##定义边界约束
param_bounds = []
for i in df.iloc[:, 0:4]:
    param_bounds.append((df[i].min(), df[i].max()))
# print(param_bounds)

In [164]:
result = minimize(
    objective,
    np.array(df.iloc[:, 0:4].mean()),
    bounds=param_bounds,
    method='SLSQP',
    options=dict(
        disp=True,
        maxiter=50,
    )
)
optimal_params = result.x
optimal_sea = -result.fun
print(optimal_params)
print(optimal_sea)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -997.2993243788926
            Iterations: 8
            Function evaluations: 44
            Gradient evaluations: 8
[32.78126497 61.77116872  1.55920182  1.3       ]
997.2993243788926
